Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A3 - Wikipedia, ORES, and Bias in Data

## Step 1⃣ | Data acquisition

We will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

First step: import the neccessary libaries for this project.

In [1]:
import pandas as pd
import os
import requests
import json
import numpy as np
from ratelimit import limits

We now load the `.csv` files as pandas dataframes for further processing.

In [74]:
population_df = pd.read_csv('../data_raw/export_2019.csv', delimiter=';')
articles_df = pd.read_csv('../data_raw/page_data.csv')

## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that we will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

Here we clean the Wikipedia articles of every row containing the `Template:` prefix.

In [75]:
articles_df = articles_df[~articles_df['page'].str.contains('Template:')]

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


### Getting article quality predictions with ORES

Now we need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For now, we only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. We now need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. We only need to capture and use the value for the prediction.

**❗Note:** It's possible that we will be unable to get a score for a particular article. If that happens, we make sure to maintain a log of articles for which we were not able to retrieve an ORES score. This log will be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

**Code for API calls**:

To retrieve the ORES prediction to an article we define the REST API endpoint.
Exchange the `User-Agent` and `From` in the header to your information if you want to run this code.

In [ ]:
####### Customize below with your own information ########
headers = {
    'User-Agent': 'https://github.com/Arne117',
    'From': 'arner92@zedat.fu-berlin.de'
}
####### Customize above with your own information ########

# 50 revisions within a given request, up to 4 parallel requests.
@limits(calls=4, period=0.1)
def get_ores_data(rev_ids, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {
        'project' : 'enwiki',
        'model'   : 'wp10',
        'revids'  : rev_ids
    }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.loads(json.dumps(response))

    return data


When we recieve the prediction from the ORES REST API we need clean the `json` data for further processing.

In [ ]:
def clean_ores_data(data):
    del data['enwiki']['models']
    chunk_df = pd.DataFrame(data['enwiki'])
    chunk_df.columns = ['score']
    chunk_df.index.name = 'rev_id'
    chunk_df['score'] = chunk_df['score'].apply(lambda score: score['wp10'])
    return chunk_df

# ❗Attention before making the requests
Due to the ORES API limitation to request a maximum of 50 IDs in one request we now chunk all IDs from the articles into 50 pieces. The requests we perform are synchronus. To not make uneccessary requests to the API you have to uncomment the following commands. Otherwise you can't proceed with the code.

In [58]:
final_ores_df = pd.DataFrame()

# The last rows index gives the length of the df and is dived by 50 to get the number of chunks needed to fit 
# 50 rev_ids into one chunk for the api call
for i, chunk in enumerate(np.array_split(articles_df['rev_id'], int(articles_df.iloc[-1].name / 50)), start=1):
    print(f"Requesting chunk {i} of {int(articles_df.iloc[-1].name / 50)}")
    chunk_rev_ids = '|'.join(map(str, chunk))
    
    ####### Uncommend below #########
    
    #chunk_ores_result = get_ores_data(chunk_rev_ids, headers)
    #chunk_ores_df = clean_ores_data(chunk_ores_result)
    #final_ores_df = pd.concat([final_ores_df, chunk_ores_df])
    
    ####### Uncommend above #########
    
    


Requesting chunk 1 of 943
Requesting chunk 2 of 943
Requesting chunk 3 of 943
Requesting chunk 4 of 943
Requesting chunk 5 of 943
Requesting chunk 6 of 943
Requesting chunk 7 of 943
Requesting chunk 8 of 943
Requesting chunk 9 of 943
Requesting chunk 10 of 943
Requesting chunk 11 of 943
Requesting chunk 12 of 943
Requesting chunk 13 of 943
Requesting chunk 14 of 943
Requesting chunk 15 of 943
Requesting chunk 16 of 943
Requesting chunk 17 of 943
Requesting chunk 18 of 943
Requesting chunk 19 of 943
Requesting chunk 20 of 943
Requesting chunk 21 of 943
Requesting chunk 22 of 943
Requesting chunk 23 of 943
Requesting chunk 24 of 943
Requesting chunk 25 of 943
Requesting chunk 26 of 943
Requesting chunk 27 of 943
Requesting chunk 28 of 943
Requesting chunk 29 of 943
Requesting chunk 30 of 943
Requesting chunk 31 of 943
Requesting chunk 32 of 943
Requesting chunk 33 of 943
Requesting chunk 34 of 943
Requesting chunk 35 of 943
Requesting chunk 36 of 943
Requesting chunk 37 of 943
Requesting

Requesting chunk 805 of 943
Requesting chunk 806 of 943
Requesting chunk 807 of 943
Requesting chunk 808 of 943
Requesting chunk 809 of 943
Requesting chunk 810 of 943
Requesting chunk 811 of 943
Requesting chunk 812 of 943
Requesting chunk 813 of 943
Requesting chunk 814 of 943
Requesting chunk 815 of 943
Requesting chunk 816 of 943
Requesting chunk 817 of 943
Requesting chunk 818 of 943
Requesting chunk 819 of 943
Requesting chunk 820 of 943
Requesting chunk 821 of 943
Requesting chunk 822 of 943
Requesting chunk 823 of 943
Requesting chunk 824 of 943
Requesting chunk 825 of 943
Requesting chunk 826 of 943
Requesting chunk 827 of 943
Requesting chunk 828 of 943
Requesting chunk 829 of 943
Requesting chunk 830 of 943
Requesting chunk 831 of 943
Requesting chunk 832 of 943
Requesting chunk 833 of 943
Requesting chunk 834 of 943
Requesting chunk 835 of 943
Requesting chunk 836 of 943
Requesting chunk 837 of 943
Requesting chunk 838 of 943
Requesting chunk 839 of 943
Requesting chunk 840

We save now the response from the API to not make all the requests again.

In [157]:
final_ores_df.to_csv('../data_raw/ores_scores_tmp.csv')

And now load it back in. To get the same data frame as before we need to transorm the json string to a python dict.

In [271]:
ores_df = pd.read_csv('../data_raw/ores_scores_tmp.csv', index_col='rev_id')
ores_df['score'] = ores_df['score'].str.replace("\'", "\"")
ores_df['score'] = ores_df['score'].apply(lambda score: json.loads(score))

### Combining the datasets

Now we need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, we'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

We remove any rows that do not have matching data, and output them to a `CSV` file called `countries-no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file looks like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

Here we combine the articles data frame with the ORES result data frame for further processing. 

In [272]:
merged_df = articles_df.join(ores_df, on='rev_id')

,page,country,rev_id,score
1,Bir I of Kanem,Chad,355319463,"{'score': {'prediction': 'Stub', 'probability'..."
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,"{'score': {'prediction': 'Stub', 'probability'..."
12,Yos Por,Cambodia,393822005,"{'score': {'prediction': 'Stub', 'probability'..."
23,Julius Gregr,Czech Republic,395521877,"{'score': {'prediction': 'Stub', 'probability'..."
24,Edvard Gregr,Czech Republic,395526568,"{'score': {'prediction': 'Stub', 'probability'..."
...,...,...,...,...
47192,Yahya Jammeh,Gambia,807482007,"{'score': {'prediction': 'GA', 'probability': ..."
47193,Lucius Fairchild,United States,807483006,"{'score': {'prediction': 'C', 'probability': {..."
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153,"{'score': {'prediction': 'GA', 'probability': ..."
47195,Francis Fessenden,United States,807483270,"{'score': {'prediction': 'C', 'probability': {..."


Now we merge the population data with the the ORES scores

In [273]:
merged_df = merged_df.merge(population_df, on='country', how='left')

,page,country,rev_id,score,population,region
0,Bir I of Kanem,Chad,355319463,"{'score': {'prediction': 'Stub', 'probability'...",16.877,AFRICA
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,"{'score': {'prediction': 'Stub', 'probability'...",5.008,ASIA
2,Yos Por,Cambodia,393822005,"{'score': {'prediction': 'Stub', 'probability'...",15.497,ASIA
3,Julius Gregr,Czech Republic,395521877,"{'score': {'prediction': 'Stub', 'probability'...",NaN,NaN
4,Edvard Gregr,Czech Republic,395526568,"{'score': {'prediction': 'Stub', 'probability'...",NaN,NaN
...,...,...,...,...,...,...
46696,Yahya Jammeh,Gambia,807482007,"{'score': {'prediction': 'GA', 'probability': ...",2.417,AFRICA
46697,Lucius Fairchild,United States,807483006,"{'score': {'prediction': 'C', 'probability': {...",329.878,NORTHERN AMERICA
46698,Fahd of Saudi Arabia,Saudi Arabia,807483153,"{'score': {'prediction': 'GA', 'probability': ...",35.041,ASIA
46699,Francis Fessenden,United States,807483270,"{'score': {'prediction': 'C', 'probability': {...",329.878,NORTHERN AMERICA


Here we remove all the the entries where ORES had no prediction and save them as `ORES_no_scores.csv`.
Also we seperate the entries where ORES returend a prediction.

In [390]:
ores_no_score_mask = merged_df['score'].apply(lambda res: 'error' in res)
ores_score_mask = merged_df['score'].apply(lambda res: 'score' in res)

ores_error_df = merged_df[ores_no_score_mask]
ores_score_df = merged_df[ores_score_mask]

ores_error_df[['page', 'country', 'rev_id', 'score']].to_csv('../data_clean/ORES_no_scores.csv', index=False)

We now proceed to remove the unused dictonary structure and format the dataframe to the required structure. The follwing code might returns a warning, but you can proceed regardless.

In [275]:
final_df = ores_score_df
final_df['article_quality'] = final_df['score'].apply(lambda entry: entry['score']['prediction'])
final_df = final_df.rename(columns={'page': 'article_name'})
final_df = final_df.drop('score', 1)
final_df = final_df[['article_name', 'country', 'region', 'rev_id', 'article_quality', 'population']]

<ipython-input-275-a470bd0abaad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['article_quality'] = final_df['score'].apply(lambda entry: entry['score']['prediction'])


Now we sort out all missing data denoted as NaN and save the entries as `countries_no_match.csv` in the `data_clean` folder.

In [ ]:
countries_no_match_df = final_df[final_df.isna().any(axis=1)]
countries_no_match_df.to_csv('../data_clean/countries_no_match.csv', index=False)

Finally, we drop all missing data from the final dataset and save it as `politicians_by_country.csv` in the  `data_clean` folder.

In [ ]:
final_df = final_df.dropna()
final_df.to_csv('../data_clean/politicians_by_country.csv', index=False)

## Step 3⃣ | Analysis

Our analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`) for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

### Results format

The results from this analysis are six `data tables`:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

These tables are embedded in the Jupyter notebook.

Load the previous saved `politicians_by_country.csv` file to perform the analysis.

In [276]:
fin_df = pd.read_csv('../data_clean/politicians_by_country.csv')
fin_df

,article_name,country,region,rev_id,article_quality,population
0,Bir I of Kanem,Chad,AFRICA,355319463,Stub,16.877
1,Information Minister of the Palestinian Nation...,Palestinian Territory,ASIA,393276188,Stub,5.008
2,Yos Por,Cambodia,ASIA,393822005,Stub,15.497
3,Robert Douglas Cook,Canada,NORTHERN AMERICA,401577829,Stub,38.190
4,List of Grand Viziers of Egypt,Egypt,AFRICA,442937236,Stub,100.803
...,...,...,...,...,...,...
44616,Hal Bidlack,United States,NORTHERN AMERICA,807481636,C,329.878
44617,Yahya Jammeh,Gambia,AFRICA,807482007,GA,2.417
44618,Lucius Fairchild,United States,NORTHERN AMERICA,807483006,C,329.878
44619,Fahd of Saudi Arabia,Saudi Arabia,ASIA,807483153,GA,35.041


#### 1. Top 10 countries by coverage
10 highest-ranked countries in terms of number of politician articles as a proportion of country population

In [324]:
coverage_df = final_df.groupby(['country']).aggregate({'population':'first', 'article_name':'count'})
coverage_df = coverage_df.rename(columns={'article_name': 'article_count'})
coverage_df['coverage'] = (coverage_df['article_count'] * 100) / (coverage_df['population'] * 1000000)

coverage_df = coverage_df.sort_values('coverage', ascending=False)
# coverage_df['coverage'].nlargest(10)
coverage_df.head(10)

,population,article_count,coverage
country,,,
Tuvalu,0.010,54,0.540000
Albania,2.838,457,0.016103
New Zealand,4.987,783,0.015701
Norway,5.387,656,0.012177
Moldova,3.535,423,0.011966
Estonia,1.331,148,0.011119
Finland,5.529,569,0.010291
Sao Tome and Principe,0.210,21,0.010000
Lithuania,2.794,244,0.008733


Save it to the `results` folder:

In [383]:
coverage_df.head(10).to_csv('../results/top_10_countries_by_coverage.csv', index=False)

#### 2. Bottom 10 countries by coverage
10 lowest-ranked countries in terms of number of politician articles as a proportion of country population

In [325]:
# coverage_df['coverage'].nsmallest(10)
coverage_df.tail(10)

,population,article_count,coverage
country,,,
Montenegro,622.0,72,0.000012
Martinique,356.0,34,0.000010
French Guiana,294.0,27,0.000009
Suriname,605.0,40,0.000007
Cape Verde,556.0,36,0.000006
Bahamas,393.0,20,0.000005
Barbados,287.0,14,0.000005
Belize,419.0,16,0.000004
Djibouti,988.0,37,0.000004


Save it to the `results` folder:

In [384]:
coverage_df.tail(10).to_csv('../results/bottom_10_countries_by_coverage.csv', index=False)

#### 3. Top 10 countries by relative quality
10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [385]:
article_quality_df = final_df[(final_df['article_quality'] == 'GA') | (final_df['article_quality'] == 'FA')]
article_quality_df = article_quality_df.drop(['article_name', 'country', 'region', 'population'], 1)
quality_df = final_df
quality_df = quality_df.merge(article_quality_df, on=['rev_id'], how='left')

quality_df = quality_df.groupby(['country']).aggregate({
    'population':'first',
    'article_quality_x':'count',
    'article_quality_y': 'count'
})

quality_df = quality_df.rename(columns={'article_quality_x': 'article_count'})
quality_df = quality_df.rename(columns={'article_quality_y': 'article_quality_count'})

quality_df['quality'] = (quality_df['article_quality_count'] * 100) / quality_df['article_count']

quality_df = quality_df.sort_values('quality', ascending=False)
quality_df.head(10)

,population,article_count,article_quality_count,quality
country,,,,
"Korea, North",25.779,36,8,22.222222
Saudi Arabia,35.041,117,15,12.820513
Romania,19.241,343,42,12.244898
Central African Republic,4.830,66,8,12.121212
Uzbekistan,34.174,28,3,10.714286
Mauritania,4.650,48,5,10.416667
Guatemala,18.066,83,7,8.433735
Dominica,72.000,12,1,8.333333
Syria,19.398,128,10,7.812500


Save it to the `results` folder:

In [386]:
quality_df.head(10).to_csv('../results/top_10_countries_by_relative_quality.csv', index=False)

#### 4. Bottom 10 countries by relative quality
10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [375]:
quality_df.tail(10)

,population,article_count,article_quality_count,quality
country,,,,
San Marino,34.000,81,0,0.0
Sao Tome and Principe,0.210,21,0,0.0
Bahrain,1.465,42,0,0.0
Guyana,787.000,20,0,0.0
Seychelles,98.000,21,0,0.0
Guadeloupe,375.000,49,0,0.0
Solomon Islands,715.000,97,0,0.0
Grenada,113.000,36,0,0.0
Cape Verde,556.000,36,0,0.0


Save it to the `results` folder:

In [387]:
quality_df.head(10).to_csv('../results/bottom_10_countries_by_relative_quality.csv', index=False)

#### 5. Regions by coverage
Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population

In [377]:
region_df = final_df.groupby(['region']).aggregate({'population':'first', 'article_name':'count'})
region_df = region_df.rename(columns={'article_name': 'article_count'})
region_df['region_coverage'] = (region_df['article_count'] * 100) / (region_df['population'] * 1000000)
region_df = region_df.sort_values('region_coverage', ascending=False)

region_df

,population,article_count,region_coverage
region,,,
AFRICA,16.877,6844,0.040552
ASIA,5.008,11691,0.233446
EUROPE,83.300,15776,0.018939
LATIN AMERICA AND THE CARIBBEAN,6.596,5273,0.079942
NORTHERN AMERICA,38.190,1910,0.005001
OCEANIA,4.987,3127,0.062703


Save it to the `results` folder:

In [388]:
region_df.to_csv('../results/regions_by_coverage.csv', index=False)

#### 6. Regions by coverage
Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

In [382]:
article_quality_df_2 = final_df[(final_df['article_quality'] == 'GA') | (final_df['article_quality'] == 'FA')]
article_quality_df_2 = article_quality_df_2.drop(['article_name', 'country', 'region', 'population'], 1)
region_quality_df = final_df
region_quality_df = region_quality_df.merge(article_quality_df_2, on=['rev_id'], how='left')
region_quality_df

region_quality_df = region_quality_df.groupby(['region']).aggregate({
    'country':'first',
    'article_quality_x':'count',
    'article_quality_y': 'count'
})

region_quality_df = region_quality_df.rename(columns={'article_quality_x': 'article_count'})
region_quality_df = region_quality_df.rename(columns={'article_quality_y': 'article_quality_count'})

region_quality_df['coverage'] = (region_quality_df['article_quality_count'] * 100) / region_quality_df['article_count']
region_quality_df = region_quality_df.sort_values('coverage', ascending=False)

region_quality_df

,country,article_count,article_quality_count,coverage
region,,,,
NORTHERN AMERICA,Canada,1910,104,5.445026
ASIA,Palestinian Territory,11691,316,2.702934
EUROPE,Germany,15776,350,2.218560
OCEANIA,New Zealand,3127,63,2.014711
AFRICA,Chad,6844,119,1.738749
LATIN AMERICA AND THE CARIBBEAN,Nicaragua,5273,76,1.441305


Save it to the `results` folder:

In [389]:
region_quality_df.to_csv('../results/regions_by_relative_article_quality.csv', index=False)

***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).